In [1]:
import pandas as pd
import geopandas as gpd

from runner import calibration, scenario

import os
from os import listdir
from os.path import isfile, join
import pathlib

#Pandas display options to show more data when DataFrames are shown in the notebook (defaults are quite limited)
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300
pd.options.display.precision = 5

#pygeos and pyogrio can speed up geopandas but can also cause package dependency conflicts - disable if conflicts occur on your system.
#gpd.options.use_pygeos = True
import pyogrio #Only works if (engine = "pyogrio") is used in geopandas .to_file() code. 

In [2]:
scenarios = ["baseline", "moderate_development", "sustainable_development"]

processed_input_files_directory = r"..\mled_processed_input_files"
processed_input_files_filenames = list(pathlib.Path(processed_input_files_directory).glob('*.csv'))
scenario_input_files_dictionary = dict(zip(scenarios, processed_input_files_filenames))

specification_files_directory = r"..\onsset_replication\onsset_input_files"
specification_files_filenames = list(pathlib.Path(specification_files_directory).glob('*.xlsx'))
scenario_specification_files_dictionary = dict(zip(scenarios, specification_files_filenames))

results_folder = r'..\results\onsset_full_results'
summary_folder = r'..\results\onsset_summary_results'
nest_summary_folder = r'..\results\nest_summaries'

In [3]:
for mled_scenario in scenarios:
    print("Running MLED scenario '" + mled_scenario +"'...")
    specs_path = os.path.join(scenario_specification_files_dictionary[mled_scenario])

    specs = pd.read_excel(specs_path, index_col=0)

    calibrated_csv_path = os.path.join(scenario_input_files_dictionary[mled_scenario])

    scenario(specs_path, calibrated_csv_path, results_folder, summary_folder, mled_scenario)

Running MLED scenario 'baseline'...
OnSSET Scenario: 1
Total electrification rate achieved in 2040 is 92.7 %
Total electrification rate achieved in 2060 is 99.8 %


2023-02-27 11:37:44,204		Finished


Running MLED scenario 'moderate_development'...
OnSSET Scenario: 1
Total electrification rate achieved in 2040 is 96.4 %
Total electrification rate achieved in 2060 is 99.9 %


2023-02-27 11:40:48,133		Finished


Running MLED scenario 'sustainable_development'...
OnSSET Scenario: 1
Total electrification rate achieved in 2040 is 100.0 %
Total electrification rate achieved in 2060 is 100.0 %


2023-02-27 11:43:51,019		Finished


In [4]:
output_files_directory = results_folder
output_files_filenames = list(pathlib.Path(output_files_directory).glob('*.csv'))
scenario_output_files_dictionary = dict(zip(scenarios, output_files_filenames))

In [5]:
full_summary = pd.DataFrame()

for mled_scenario in scenarios:
    
    nest = pd.read_csv(os.path.join(scenario_output_files_dictionary[mled_scenario]))

    nest["BCU"] = nest["BCU"].astype("int")
    nest[["BCU","FinalElecCode2020","isurban_2020", "FinalElecCode2040","isurban_future_2040", "FinalElecCode2060","isurban_future_2060"]] = nest[["BCU","FinalElecCode2020","isurban_2020", "FinalElecCode2040","isurban_future_2040", "FinalElecCode2060","isurban_future_2060"]].astype("category")
    nest["isurban_future_2020"] = nest["isurban_2020"]

    nest.FinalElecCode2020 = nest.FinalElecCode2020.cat.set_categories([1,2,3,5,6,7,99])
    nest.FinalElecCode2040 = nest.FinalElecCode2040.cat.set_categories([1,2,3,5,6,7,99])
    nest.FinalElecCode2060 = nest.FinalElecCode2060.cat.set_categories([1,2,3,5,6,7,99])

    nest.FinalElecCode2020 = nest.FinalElecCode2020.cat.rename_categories({1: "grid_existing", 2: "grid_new", 3: "sa_pv", 5: "mg_pv_hybrid", 6: "mg_wind_hybrid", 7: "mg_hydro", 99: "unelectrified"})
    nest.FinalElecCode2040 = nest.FinalElecCode2040.cat.rename_categories({1: "grid_existing", 2: "grid_new", 3: "sa_pv", 5: "mg_pv_hybrid", 6: "mg_wind_hybrid", 7: "mg_hydro", 99: "unelectrified"})
    nest.FinalElecCode2060 = nest.FinalElecCode2060.cat.rename_categories({1: "grid_existing", 2: "grid_new", 3: "sa_pv", 5: "mg_pv_hybrid", 6: "mg_wind_hybrid", 7: "mg_hydro", 99: "unelectrified"})

    nest.isurban_2020 = nest.isurban_2020.cat.rename_categories({0: "rural", 1: "urban"})
    nest.isurban_future_2020 = nest.isurban_future_2020.cat.rename_categories({0: "rural", 1: "urban"})
    nest.isurban_future_2040 = nest.isurban_future_2040.cat.rename_categories({0: "rural", 1: "urban"})
    nest.isurban_future_2060 = nest.isurban_future_2060.cat.rename_categories({0: "rural", 1: "urban"})

    years = [2020, 2040, 2060]
    scenario_summary = pd.DataFrame()

    for year in years:
        yearly_summary = ((nest.groupby(["BCU","FinalElecCode"+str(year),"isurban_future_"+str(year)])["tot_dem_"+str(year)].sum()/10**6).round(2)).to_frame().reset_index()
        yearly_summary = yearly_summary.set_axis(["BCU", "tec", "urb_rur","value"], axis = 1)
        yearly_summary["year"] = year
        yearly_summary["scenario"] = mled_scenario
        yearly_summary["unit"] = "GWh"
        yearly_summary.insert(yearly_summary.shape[1]-1, "value", yearly_summary.pop("value"))

        scenario_summary = pd.concat([scenario_summary, yearly_summary])
        
    full_summary = pd.concat([full_summary, scenario_summary])
    
full_summary.to_csv(os.path.join(nest_summary_folder, "energy_allocation_results_for_nest.csv"), index = False)

In [6]:
full_summary

,BCU,tec,urb_rur,year,scenario,unit,value
0,1,grid_existing,rural,2020,baseline,GWh,5.19
1,1,grid_existing,urban,2020,baseline,GWh,0.00
2,1,grid_new,rural,2020,baseline,GWh,0.00
3,1,grid_new,urban,2020,baseline,GWh,0.00
4,1,sa_pv,rural,2020,baseline,GWh,0.00
...,...,...,...,...,...,...,...
331,24,mg_wind_hybrid,urban,2060,sustainable_development,GWh,0.00
332,24,mg_hydro,rural,2060,sustainable_development,GWh,0.00
333,24,mg_hydro,urban,2060,sustainable_development,GWh,0.00
334,24,unelectrified,rural,2060,sustainable_development,GWh,0.00
